In [1]:
# %load ../import_data.py
from collections import Counter
import pandas as pd
import numpy as np
import itertools
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)

import re
import xgboost as xgb
import joblib 
from sklearn.model_selection import train_test_split
    
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from statistic import Statistic
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction
from helper import DataHelper

# for autoreload modules
%load_ext autoreload
%autoreload 2

In [2]:
import category_encoders as ce

In [3]:
country_code = pd.read_csv(UtilsKy.COUNTY_CODE, dtype=str,skipinitialspace=True)
country_code.columns = ['Country_full', 'country_code_2', 'country', 'country_code']
country_code = country_code[['country', 'country_code']].copy()

In [5]:
db_train_ky10 = pd.read_csv(UtilsKy.DB_TEACH_KY10, dtype=str)
db_test_ky10 = pd.read_csv(UtilsKy.DB_TEST_KY10, dtype=str)

In [22]:
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)
white = pd.read_csv(UtilsKy.WHITE_KY9_FOR_PROD , dtype=str)

In [23]:
resolved = pd.read_csv(UtilsKy.KY_10_RESOLVED, dtype=str)

In [24]:
resolved.head(3)

,amount,amount_deviation_usd,amount_usd,area_code,bank_currency,bank_name,bin,card_type,city,client_hour,continent_code,count_months_to_end_card,country,country_code3,currency,date,date_cb_only,date_only,day_of_week,distance_haversine,distance_l2,distance_sqs_subtraction,hour,id,is_bank_country_equal_country,is_city_resolved,is_ip_country_equal_country,latitude,longitude,metro_code,order_id,phone_2_norm,region_name,status
0,141.68,0,141.68,860,840,"TD BANK, N.A.",448233,VISA card,Vernon Rockville,14,NaN,14,USA,USA,USD,2021-02-02 21:10:22,undefined,2021-02-02,2,0.153902149788471,0.403481497683806,4.18798475693855,21,448233******3385_2022-04,1,1,1,41.8365,-72.4632,533,58963204904455627,60,Connecticut,0
1,26.83,-2,26.83,0,840,NASSAU EDUCATORS F.C.U.,438787,VISA card,undefined,21,NaN,45,USA,USA,USD,2021-02-02 21:11:20,undefined,2021-02-02,2,0.0140942632758067,0.033064662702264,7.74357209333864,21,438787******8595_2024-11,1,0,1,38.0000,-97.0000,0,58963219744750136,16,undefined,0
2,34.77,-2,34.77,0,840,"BANK OF AMERICA, N.A.",481582,VISA card,undefined,21,NaN,35,USA,USA,USD,2021-02-02 21:14:03,undefined,2021-02-02,2,0.0140942632758067,0.033064662702264,7.74357209333864,21,481582******0912_2024-01,1,0,1,38.0000,-97.0000,0,58963261615776513,14,undefined,0


In [25]:
# metro_decode factor
resolved['metro_decode'] = np.where((resolved.metro_code == '803') | (resolved.metro_code == '501'), '1', resolved.metro_code)
resolved['metro_decode'] = np.where((resolved.metro_decode != '0') & (resolved.metro_decode != '1'), '2', resolved.metro_decode)

In [26]:
resolved.metro_decode.value_counts()

0    99330
2    89726
1    17050
Name: metro_decode, dtype: int64

In [75]:
db_teach, db_test = Statistic.split_train_test_with_diff_ids(resolved, train_size=0.8) # test_has_unique_ids=True
print(db_teach.status.value_counts())
print(db_test.status.value_counts())
# For Xgboost

COL_FACTORS = ['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'longitude', 'latitude', 'phone_2_norm', 'is_city_resolved',] #  metro_decode
# 'is_gender_undefined'
COL_FACTORS = sorted(COL_FACTORS)
datahelper = DataHelper(db_teach, db_test, COL_FACTORS)
datahelper.create_train_test()
datahelper.show_columns_with_na()
mean_values = datahelper.get_mean_value()
replaced_values = { col: mean_values[col] for col in ('latitude', 'longitude')}
replaced_values['default'] =  -999
datahelper.replaced_na_values(replaced_values)   
train , test = datahelper.get_train_test()
train_test = pd.concat([train, test])
train_test.sort_index(inplace=True)

0    163765
1      1110
Name: status, dtype: int64
0    40942
1      289
Name: status, dtype: int64
Statistic na values in columns : 
train na columns : Index(['latitude', 'longitude'], dtype='object')
latitude_na: 855,longitude_na: 855
test na columns : Index(['latitude', 'longitude'], dtype='object')
latitude_na: 204,longitude_na: 204
Replaced na values:
is_city_resolved_na -> -999
latitude_na -> 33.26687733020365
bank_currency_na -> -999
bin_na -> -999
day_of_week_na -> -999
phone_2_norm_na -> -999
hour_na -> -999
longitude_na -> -77.99604855749298
amount_na -> -999


In [63]:
max_depths = [5]
n_rounds = [80, 90, 100]
learning_rates = [0.05, 0.1, 0.15]

max_depths = [5]
n_rounds = [90]
learning_rates = [0.1]

In [64]:
random_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] + [11, 12, 13, 14, 15, 16, 17, 18, 19, 20] + [21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
# + [2387, 84, 22281, 440, 710]

In [76]:
results = []
for rand in random_list:
    train_index, test_index = Statistic.split_train_test_indexes(resolved, train_size=0.8, random_state=rand, test_has_unique_ids=False)
    
    db_teach, db_test = resolved.loc[train_index, :], resolved.loc[test_index, :]
    train, test = train_test.loc[train_index, :], train_test.loc[test_index, :]
    
    label = np.where(Statistic.is_status_bad(db_teach), 1, 0)
    analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white )#
    result_df_amount = None
    scale_pos_weight = round(len(db_teach)/len(Statistic.get_dt_bad(db_teach)), 3)
    for learning_rate, max_depth, n_round in itertools.product(learning_rates, max_depths, n_rounds):
        config = {'max_depth': max_depth, 'learning_rate': learning_rate, 'n_estimators': n_round}
        model = xgb.XGBClassifier(**config, use_label_encoder=False, scale_pos_weight=scale_pos_weight, eval_metric='logloss')
        model.fit(train, label)
        test_pred = model.predict_proba(test)
        db_test["probability"] = test_pred[:, 1]
        description = '-' . join([str(elem) for elem in (max_depth, n_round, learning_rate)])                    
        result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")        
    results.append(result_df_amount)

In [77]:
result_df = pd.concat(results)
n = result_df.shape[0]
result_df.index = range(n)
sub_rows = list(range(n))[::2]
result_df_without_threshold = result_df.copy().iloc[sub_rows,:]
row = {'description': 'Total'}
for col in list(result_df_without_threshold):
    if col.startswith('p_') or col == 'rating':
        row[col] = result_df_without_threshold[col].astype(float).mean()
result_df_without_threshold = result_df_without_threshold.append([row], ignore_index=True)

#### 5-90-0.1

In [52]:
# main factors from resolved_ky10 - last model in loop 70 -30
data_st = {'features': COL_FACTORS, 'rating': model.feature_importances_}
pd.DataFrame(data=data_st).sort_values(by=['rating'], ascending=False)

,features,rating
0,amount,0.140412
1,bank_currency,0.132681
8,phone_2_norm,0.124678
2,bin,0.123493
7,longitude,0.109803
6,latitude,0.104497
3,day_of_week,0.100230
5,is_city_resolved,0.092460
4,hour,0.071745


In [53]:
# main factors from resolved_ky10 - 30 folds 70 -30
mask = result_df_without_threshold.description != 'Total'
g = result_df_without_threshold[mask].groupby(['description'])['rating'].mean()
best_param = pd.DataFrame(g, columns=['rating'])
best_param.sort_values(by='rating', ascending=False, inplace=True)
best_param

,rating
description,
5-90-0.1,163.723667


In [54]:
# main factors from resolved_ky10 - 30 folds 70 -30
result_df_without_threshold.tail(15)

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating,n_white_list,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
16,5-90-0.1,4.630000,7.540000,9.910000,13.960,14.090000,14.090,15.460,20.430000,33.420000,133.530000,1055452.0,10424.0,16.0,1045857.17,2376.23,144327.0,1033.0,61779.0,366.0,69692.98,6885047.30
17,5-90-0.1,1.530000,3.440000,10.100000,13.050,17.150000,20.020,23.300,29.320000,43.250000,161.160000,1055452.0,10325.0,10.0,1011884.71,1252.53,144246.0,952.0,61860.0,447.0,85274.05,6940540.02
18,5-90-0.1,0.720000,4.040000,8.210000,10.450,15.240000,17.080,19.180,22.100000,36.370000,133.390000,1055452.0,10239.0,40.0,1007127.45,3618.09,144287.0,993.0,61819.0,406.0,87495.61,7014270.61
19,5-90-0.1,3.970000,8.440000,14.100000,16.030,19.890000,22.580,26.110,28.730000,45.220000,185.070000,1055452.0,10402.0,28.0,1033700.42,3582.53,144260.0,966.0,61846.0,433.0,84602.27,6942585.68
20,5-90-0.1,7.800000,12.650000,17.960000,21.840,24.360000,27.680,29.980,36.750000,49.340000,228.360000,1055452.0,10278.0,12.0,1056457.02,1253.57,144262.0,968.0,61844.0,431.0,82619.69,6950788.29
21,5-90-0.1,4.660000,7.600000,8.610000,9.680,14.160000,17.360,22.870,25.340000,53.620000,163.900000,1055452.0,10224.0,27.0,1009369.43,2731.12,144311.0,1017.0,61795.0,382.0,79367.95,6883951.27
22,5-90-0.1,10.670000,12.770000,14.350000,15.420,17.290000,21.310,23.790,26.910000,40.600000,183.110000,1055452.0,10324.0,31.0,1034945.93,3206.35,144251.0,957.0,61855.0,442.0,78802.93,6924114.49
23,5-90-0.1,9.180000,11.140000,11.580000,12.560,13.880000,15.310,16.320,19.960000,35.660000,145.590000,1055452.0,10291.0,38.0,1042221.34,4893.39,144265.0,971.0,61841.0,428.0,70843.33,6913595.58
24,5-90-0.1,4.570000,7.100000,8.830000,10.930,15.530000,17.660,18.510,22.320000,35.220000,140.670000,1055452.0,10365.0,25.0,1041547.17,2371.78,144283.0,989.0,61823.0,410.0,81101.33,6910941.38
25,5-90-0.1,4.110000,12.870000,17.800000,19.320,21.890000,22.410,23.680,33.080000,48.910000,204.070000,1055452.0,10281.0,21.0,1015886.95,2959.15,144324.0,1030.0,61782.0,369.0,72083.28,6919469.92


In [78]:
# main factors from resolved_ky10 - 30 folds - 80 -20
mask = result_df_without_threshold.description != 'Total'
g = result_df_without_threshold[mask].groupby(['description'])['rating'].mean()
best_param = pd.DataFrame(g, columns=['rating'])
best_param.sort_values(by='rating', ascending=False, inplace=True)
best_param

,rating
description,
5-90-0.1,168.485333


In [79]:
# main factors from resolved_ky10 - 30 folds 80 -20
result_df_without_threshold.tail(15)

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating,n_white_list,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
16,5-90-0.1,4.800,6.700000,9.610000,9.830,10.240,10.350000,13.940000,23.450,36.800000,125.720000,1055452.0,6918.0,8.0,678233.98,934.02,164911.0,1146.0,41195.0,253.0,50065.75,4566644.14
17,5-90-0.1,6.860,11.620000,15.940000,21.020,22.790,24.900000,24.900000,35.300,52.600000,215.930000,1055452.0,6937.0,8.0,677852.14,1055.01,164870.0,1105.0,41236.0,294.0,53583.31,4637746.31
18,5-90-0.1,6.840,7.710000,8.710000,10.330,15.050,17.280000,19.410000,22.020,36.570000,143.920000,1055452.0,6837.0,25.0,670139.90,2138.11,164882.0,1117.0,41224.0,282.0,63780.48,4674073.57
19,5-90-0.1,9.770,12.970000,14.430000,16.450,17.270,19.770000,20.980000,27.040,40.470000,179.150000,1055452.0,6788.0,15.0,665696.50,1399.25,164853.0,1088.0,41253.0,311.0,52784.12,4584903.96
20,5-90-0.1,9.600,11.610000,14.990000,19.010,21.620,24.760000,25.780000,29.990,51.590000,208.950000,1055452.0,6937.0,7.0,729019.89,612.53,164885.0,1120.0,41221.0,279.0,53144.03,4646030.49
21,5-90-0.1,2.820,6.580000,8.820000,11.720,14.540,22.960000,24.450000,36.810,57.090000,185.790000,1055452.0,6793.0,10.0,682256.63,1120.08,164930.0,1165.0,41176.0,234.0,57147.26,4585097.86
22,5-90-0.1,10.570,13.000000,14.400000,15.810,17.510,20.220000,21.710000,29.920,41.100000,184.240000,1055452.0,6880.0,15.0,683564.26,1563.26,164850.0,1085.0,41256.0,314.0,57690.71,4642073.95
23,5-90-0.1,11.350,13.980000,14.090000,15.420,15.940,17.340000,17.870000,23.030,40.690000,169.710000,1055452.0,6786.0,32.0,702851.02,4133.48,164856.0,1091.0,41250.0,308.0,50676.32,4611503.19
24,5-90-0.1,7.030,7.920000,9.710000,12.720,18.280,18.600000,19.230000,25.800,44.310000,163.600000,1055452.0,6843.0,12.0,687507.12,1129.60,164917.0,1152.0,41189.0,247.0,48020.25,4618131.98
25,5-90-0.1,11.390,17.780000,19.030000,26.220,27.690,28.630000,29.440000,35.710,42.060000,237.950000,1055452.0,6755.0,18.0,661819.69,2343.19,164903.0,1138.0,41203.0,261.0,50890.84,4605298.48


In [45]:
# with metro_decode from resolved_ky10 - last model in loop 70 -30
data_st = {'features': COL_FACTORS, 'rating': model.feature_importances_}
pd.DataFrame(data=data_st).sort_values(by=['rating'], ascending=False)

,features,rating
0,amount,0.130745
1,bank_currency,0.113148
2,bin,0.107121
9,phone_2_norm,0.106731
7,longitude,0.099103
6,latitude,0.097629
8,metro_decode,0.097475
5,is_city_resolved,0.093748
3,day_of_week,0.086035
4,hour,0.068264


In [46]:
# with metro_decode from resolved_ky10 - 30 folds 70 -30
mask = result_df_without_threshold.description != 'Total'
g = result_df_without_threshold[mask].groupby(['description'])['rating'].mean()
best_param = pd.DataFrame(g, columns=['rating'])
best_param.sort_values(by='rating', ascending=False, inplace=True)
best_param

,rating
description,
5-90-0.1,164.414333


In [61]:
# with metro_decode from resolved_ky10 - 30 folds 70 -30
result_df_without_threshold.tail(15)

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating,n_white_list,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
16,5-90-0.1,7.100000,9.090,9.520000,11.210000,13.42,15.760000,16.980,19.540000,27.980,130.600000,1055452.0,10424.0,16.0,1045857.17,2376.23,144327.0,1033.0,61779.0,366.0,69692.98,6885047.30
17,5-90-0.1,4.130000,6.510,12.080000,13.590000,17.67,20.050000,24.390,28.870000,46.980,174.270000,1055452.0,10325.0,10.0,1011884.71,1252.53,144246.0,952.0,61860.0,447.0,85274.05,6940540.02
18,5-90-0.1,0.000000,3.200,3.700000,9.310000,12.57,15.970000,18.490,22.830000,32.010,118.080000,1055452.0,10239.0,40.0,1007127.45,3618.09,144287.0,993.0,61819.0,406.0,87495.61,7014270.61
19,5-90-0.1,2.980000,6.460,10.060000,12.840000,13.76,18.660000,24.080,30.810000,48.340,167.990000,1055452.0,10402.0,28.0,1033700.42,3582.53,144260.0,966.0,61846.0,433.0,84602.27,6942585.68
20,5-90-0.1,7.600000,12.400,18.540000,22.590000,24.64,30.610000,34.390,39.180000,52.240,242.190000,1055452.0,10278.0,12.0,1056457.02,1253.57,144262.0,968.0,61844.0,431.0,82619.69,6950788.29
21,5-90-0.1,4.150000,10.530,16.070000,18.130000,20.78,23.030000,26.140,31.750000,49.800,200.380000,1055452.0,10224.0,27.0,1009369.43,2731.12,144311.0,1017.0,61795.0,382.0,79367.95,6883951.27
22,5-90-0.1,9.040000,12.170,14.280000,16.330000,16.90,18.610000,19.500,29.320000,41.290,177.440000,1055452.0,10324.0,31.0,1034945.93,3206.35,144251.0,957.0,61855.0,442.0,78802.93,6924114.49
23,5-90-0.1,9.750000,10.130,11.970000,12.390000,12.74,14.430000,16.140,22.050000,37.850,147.450000,1055452.0,10291.0,38.0,1042221.34,4893.39,144265.0,971.0,61841.0,428.0,70843.33,6913595.58
24,5-90-0.1,3.910000,5.730,7.940000,11.470000,14.13,16.540000,20.170,24.440000,34.730,139.060000,1055452.0,10365.0,25.0,1041547.17,2371.78,144283.0,989.0,61823.0,410.0,81101.33,6910941.38
25,5-90-0.1,13.120000,15.930,20.670000,22.710000,25.38,27.920000,28.680,34.140000,46.550,235.100000,1055452.0,10281.0,21.0,1015886.95,2959.15,144324.0,1030.0,61782.0,369.0,72083.28,6919469.92


In [73]:
# with metro_decode from resolved_ky10 - 30 folds - 80 - 20
mask = result_df_without_threshold.description != 'Total'
g = result_df_without_threshold[mask].groupby(['description'])['rating'].mean()
best_param = pd.DataFrame(g, columns=['rating'])
best_param.sort_values(by='rating', ascending=False, inplace=True)
best_param

,rating
description,
5-90-0.1,169.662


In [74]:
# with metro_decode from resolved_ky10 - 30 folds - 80 - 20
result_df_without_threshold.tail(15)

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating,n_white_list,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
16,5-90-0.1,5.010,8.410000,9.880,9.880000,12.000,13.610,14.460,19.040000,34.970000,127.260,1055452.0,6918.0,8.0,678233.98,934.02,164911.0,1146.0,41195.0,253.0,50065.75,4566644.14
17,5-90-0.1,4.890,9.910000,16.950,18.160000,23.570,26.660,30.520,35.060000,50.400000,216.120,1055452.0,6937.0,8.0,677852.14,1055.01,164870.0,1105.0,41236.0,294.0,53583.31,4637746.31
18,5-90-0.1,5.510,7.840000,7.840,10.030000,12.910,13.750,18.210,21.850000,40.590000,138.530,1055452.0,6837.0,25.0,670139.90,2138.11,164882.0,1117.0,41224.0,282.0,63780.48,4674073.57
19,5-90-0.1,5.370,10.870000,11.910,14.130000,16.700,20.240,20.850,26.570000,42.610000,169.250,1055452.0,6788.0,15.0,665696.50,1399.25,164853.0,1088.0,41253.0,311.0,52784.12,4584903.96
20,5-90-0.1,10.010,17.340000,24.720,26.660000,28.660,34.690,35.500,40.380000,50.270000,268.230,1055452.0,6937.0,7.0,729019.89,612.53,164885.0,1120.0,41221.0,279.0,53144.03,4646030.49
21,5-90-0.1,4.320,7.880000,9.480,17.860000,26.870,27.440,28.570,34.060000,53.780000,210.260,1055452.0,6793.0,10.0,682256.63,1120.08,164930.0,1165.0,41176.0,234.0,57147.26,4585097.86
22,5-90-0.1,11.890,14.150000,15.690,17.450000,19.470,19.710,21.320,31.430000,40.670000,191.780,1055452.0,6880.0,15.0,683564.26,1563.26,164850.0,1085.0,41256.0,314.0,57690.71,4642073.95
23,5-90-0.1,11.640,13.650000,14.100,15.680000,19.680,19.680,20.890,24.960000,37.890000,178.170,1055452.0,6786.0,32.0,702851.02,4133.48,164856.0,1091.0,41250.0,308.0,50676.32,4611503.19
24,5-90-0.1,4.840,6.180000,12.190,21.200000,23.840,25.630,27.350,35.340000,44.630000,201.200,1055452.0,6843.0,12.0,687507.12,1129.60,164917.0,1152.0,41189.0,247.0,48020.25,4618131.98
25,5-90-0.1,6.660,15.170000,21.410,21.700000,23.150,29.130,29.970,33.760000,47.870000,228.820,1055452.0,6755.0,18.0,661819.69,2343.19,164903.0,1138.0,41203.0,261.0,50890.84,4605298.48
